In [121]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import urllib.request, json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
READ_API_KEY = replace_your_read_api_key
CHANNEL_ID = replace_your_read_channel_ID


def get_thingspeak_data():
    conn = urllib.request.urlopen("http://api.thingspeak.com/channels/%s/feed.json?api_key=%s"% (CHANNEL_ID, READ_API_KEY))
    response = conn.read()
    print("http status code=%s" % (conn.getcode()))
    data = json.loads(response)
    feeds = data['feeds']
    conn.close()
    return feeds


def read_csv_file(json_feed):
    rows = []
    for feed in json_feed:
        fault_label = majority_based_labeling(float(feed['field1']), float(feed['field2']), float(feed['field3']))
        rows.append(fault_label)

    y = len(rows)
    fault_label_matrix = np.array(rows)
    np.reshape(fault_label_matrix, (y, 1))
    return fault_label_matrix


def calculate_vote(t1, t2, t3):
    fault_vote = []
    dif12 = abs(t1 - t2)
    dif13 = abs(t1 - t3)
    dif23 = abs(t2 - t3)

    if dif12 > 1.5:
        fault_vote.append(1)
        fault_vote.append(2)
    if dif13 > 1.5:
        fault_vote.append(1)
        fault_vote.append(3)
    if dif23 > 1.5:
        fault_vote.append(2)
        fault_vote.append(3)
    # print(fault_vote)
    return fault_vote


# Boyer-Moore voting
def findMajority(arr):
    n = len(arr)
    fault_votes = sorted(arr)
    # print(fault_votes)
    candidate = -1
    votes = 0

    # Finding majority candidate
    for i in range(n):
        if votes == 0:
            candidate = fault_votes[i]
            votes = 1
        else:
            if fault_votes[i] == candidate:
                votes += 1
            else:
                votes -= 1
    count = 0
    # print(candidate)
    # Checking if majority candidate occurs more than n/2
    # times
    for i in range(n):
        if fault_votes[i] == candidate:
            count += 1

    if n > 2 and count >= n // 2:
        return candidate
    else:
        return -1


def majority_based_labeling(t1, t2, t3):
    label = 0
    faulty_node_votes = calculate_vote(t1, t2, t3)
    # print(faulty_node_votes)
    faulty_node = findMajority(faulty_node_votes)
    # print(faulty_node)
    if faulty_node in [1, 2, 3]:
        label = faulty_node
    elif faulty_node == -1:
        label = 4
#         print("faulty node is {}".format(label))
    else:
        print("Error in fault detection")
    return label

In [123]:
data = pd.read_csv("feed.csv")
X = data.iloc[:, 2:].values
# print(np.shape(X))
# print(X)
ts_json_data = get_thingspeak_data()
Y1 = read_csv_file(ts_json_data)
Y = np.reshape(Y1, (100, 1))

http status code=200


In [148]:
ohe = OneHotEncoder()
transformed1 = ohe.fit_transform(Y)
# tf.keras.layers.Lambda(tf.sparse.to_dense)(transformed)
transformed = transformed1.toarray() 
print(np.shape(transformed))

(100, 3)


In [149]:
X_train, X_test, Y_train, Y_test = train_test_split(X, transformed, test_size=0.1, random_state=42)
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(Y_train))
print(np.shape(Y_test))

(90, 3)
(10, 3)
(90, 3)
(10, 3)


In [150]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [151]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))
ann.add(tf.keras.layers.Dense(units=3,activation="softmax"))
ann.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])

In [152]:
ann.fit(X_train,Y_train,batch_size=32,epochs = 100)

Epoch 1/100
3/3 [==============================] - 1s 7ms/step - loss: 1.1324 - accuracy: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 0s 4ms/step - loss: 1.1245 - accuracy: 0.0222
Epoch 3/100
3/3 [==============================] - 0s 4ms/step - loss: 1.1167 - accuracy: 0.0667
Epoch 4/100
3/3 [==============================] - 0s 3ms/step - loss: 1.1092 - accuracy: 0.1667
Epoch 5/100
3/3 [==============================] - 0s 4ms/step - loss: 1.1018 - accuracy: 0.2222
Epoch 6/100
3/3 [==============================] - 0s 4ms/step - loss: 1.0945 - accuracy: 0.3000
Epoch 7/100
3/3 [==============================] - 0s 4ms/step - loss: 1.0874 - accuracy: 0.3333
Epoch 8/100
3/3 [==============================] - 0s 4ms/step - loss: 1.0805 - accuracy: 0.4333
Epoch 9/100
3/3 [==============================] - 0s 5ms/step - loss: 1.0736 - accuracy: 0.5333
Epoch 10/100
3/3 [==============================] - 0s 4ms/step - loss: 1.0668 - accuracy: 0.5556
Epoch 11/100
3/3 [=======

3/3 [==============================] - 0s 4ms/step - loss: 0.4029 - accuracy: 0.9222
Epoch 85/100
3/3 [==============================] - 0s 3ms/step - loss: 0.3981 - accuracy: 0.9222
Epoch 86/100
3/3 [==============================] - 0s 4ms/step - loss: 0.3925 - accuracy: 0.9222
Epoch 87/100
3/3 [==============================] - 0s 4ms/step - loss: 0.3881 - accuracy: 0.9222
Epoch 88/100
3/3 [==============================] - 0s 3ms/step - loss: 0.3829 - accuracy: 0.9222
Epoch 89/100
3/3 [==============================] - 0s 3ms/step - loss: 0.3783 - accuracy: 0.9222
Epoch 90/100
3/3 [==============================] - 0s 3ms/step - loss: 0.3739 - accuracy: 0.9222
Epoch 91/100
3/3 [==============================] - 0s 4ms/step - loss: 0.3697 - accuracy: 0.9222
Epoch 92/100
3/3 [==============================] - 0s 4ms/step - loss: 0.3652 - accuracy: 0.9222
Epoch 93/100
3/3 [==============================] - 0s 3ms/step - loss: 0.3610 - accuracy: 0.9222
Epoch 94/100
3/3 [===============